In [130]:
!pip install inline

import inline
import matplotlib
# %matplotlib inline
import numpy as np
import cv2
# import os
# import globS
#import pydicom as dicom
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
from PIL import Image
import os, sys
import glob

path = "C:/Users/Manoranga/FYP/Segmented Sets/chest_xray/test/NORMAL"
dirs = os.listdir( path )
imageList =[]
segmentedImage =[]
segmentedImage1 =[]


def grayscale_convert(image):
    # grab the image dimensions
    h = image.shape[0]
    w = image.shape[1]

    # loop over the image, pixel by pixel
    for y in range(0, h):
        for x in range(0, w):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] == 1 else 0

    # return the grayscale image
    return image



for item in glob.glob('C:/Users/Manoranga/FYP/Segmented Sets/chest_xray/test/NORMAL/New folder/*.JPEG'):
    img = Image.open(item)
    imageList.append(img)
    
len(imageList)    
for initial in imageList:
    dim = (1024, 1024)
    initial = initial.resize((1024,1024))
    initial = np.array(initial).astype(np.uint8)
    
    gray1 = cv2.resize(initial, dim, interpolation = cv2.INTER_AREA) 
   # gray1 = cv2.cvtColor(initial, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.equalizeHist(initial)
            # median filter
    gray1 = cv2.medianBlur(gray1, 5)
            #plt.imshow( gray1)
            # noise removal
    gray1 = cv2.GaussianBlur(gray1, (5, 5), 0)
    #plt.hist(gray1.ravel(), 256)
            
#plt.show()
# Threshold the image to binary using local method
    thresh1 = cv2.adaptiveThreshold(gray1, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 699, 6)
#plt.imshow( thresh1)
    constant = cv2.rectangle(thresh1, (0, 0), (1024, 1024), (255, 255, 255), 5)
            # finding cotours
    cnts, hierachy = cv2.findContours(constant, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #print(cnts)
            #print(constant.shape)
    draw_counters = initial.copy()
    cv2.drawContours(draw_counters, cnts, -1, (0, 255, 0), -1)
            #plt.imshow(draw_counters)
            # calculate number of pixel clusters
    ret, markers = cv2.connectedComponents(constant)
            #print(markers)
    label_hue = np.uint8(179 * markers / np.max(markers))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
            # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
            # set bg label to black
    labeled_img[label_hue == 0] = 0
            #plt.imshow(labeled_img)
            # Get the area taken by each component. Ignore label 0 since this is the background.
    marker_area = [np.sum(markers == m) for m in range(np.max(markers)) if m != 0]
            #print(marker_area)
            # Get label of largest component by area
    sorted_marker_area = sorted(marker_area, reverse=True)
            #print(marker_area)
    largest_component_1 = marker_area.index(sorted_marker_area[1])
    largest_component_2 = marker_area.index(sorted_marker_area[2])
            #print(largest_component_1)
            # Get pixels which correspond to the lung
    lung_mask = markers == largest_component_1 + 1
    lung_mask = lung_mask + (markers == largest_component_2 + 1)
            #print(lung_mask)
            # lung_mask.astype(np.uint8)
    lung_mask = grayscale_convert(np.uint8(lung_mask))
            #print(lung_mask.shape)
            #plt.imshow(lung_mask)
    kernel = np.ones((32, 32), np.uint8)
    closing = cv2.morphologyEx(lung_mask, cv2.MORPH_DILATE, kernel)
            # ShowImage('Closing', closing, 'gray')
            #plt.imshow( closing)
            #
# median filter
    median = cv2.medianBlur(closing, 11)
            # converting back to binary
    ret, thresh1 = cv2.threshold(median, 127, 255, cv2.THRESH_BINARY)
    lung_out = gray1.copy()
            # In a copy of the original image, clear those pixels that don't correspond to the lungs
    lung_out[thresh1 == False] = (0)
            # ShowImage('Connected Components',lung_out,'rgb')
            #plt.imshow(lung_out)
    outAsFloat = cv2.cvtColor(lung_out, cv2.CV_32F)
    Gabor = outAsFloat.copy()
    kernal = cv2.getGaborKernel((30, 30), 2, 10, 2, 2)
            #plt.imshow(Gabor)
    
    cv2.filter2D(outAsFloat, cv2.CV_32F, kernal, Gabor)
    segmentedImage.append(Gabor)
#len(segmentedImage)

# for i in range(8):
#     cv2.imwrite('{}{}{}'.format('C:/Users/Manoranga/FYP/Segmented Sets/NewImages/',i+1,'.jpg'),Gabor)
    

# segmentedImage.append(Gabor)
# #Gabor.save(f + ' resized.jpg', 'JPEG', quality=90)

#     cv2.imwrite('{}{}{}'.format('C:/Users/Manoranga/FYP/Segmented Sets/NewImages',i+1,'.jpg'),i)
#     # new.save('{}{}{}'.format('C:/Users/Manoranga/FYP/Segmented Sets/NewImages',i+1,'.jpg'))
#     #cv2.imwrite( "C:/Users/Manoranga/FYP/Segmented Sets/NewImages",i+1,".jpg", i );


          
 












In [131]:

for num,images in enumerate(segmentedImage):
    cv2.imwrite('{}{}{}'.format('C:/Users/Manoranga/FYP/Segmented Sets/NewImages/test/NORMAL/',num+1,'.jpg'),images)
    


In [124]:
len(segmentedImage)

4